In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,fffe31003300320037003900,2008-03-10,Female,Product,Yes,2.0,NaN,5.8,0.49
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22750 entries, 0 to 22749
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           22750 non-null  object 
 1   Date of Joining       22750 non-null  object 
 2   Gender                22750 non-null  object 
 3   Company Type          22750 non-null  object 
 4   WFH Setup Available   22750 non-null  object 
 5   Designation           22750 non-null  float64
 6   Resource Allocation   21369 non-null  float64
 7   Mental Fatigue Score  20633 non-null  float64
 8   Burn Rate             21626 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.6+ MB


In [4]:
df.describe()

,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
count,22750.000000,21369.000000,20633.000000,21626.000000
mean,2.178725,4.481398,5.728188,0.452005
std,1.135145,2.047211,1.920839,0.198226
min,0.000000,1.000000,0.000000,0.000000
25%,1.000000,3.000000,4.600000,0.310000
50%,2.000000,4.000000,5.900000,0.450000
75%,3.000000,6.000000,7.100000,0.590000
max,5.000000,10.000000,10.000000,1.000000


In [5]:
m=df.isna().sum()
m[m>0]

Resource Allocation     1381
Mental Fatigue Score    2117
Burn Rate               1124
dtype: int64

In [59]:
df.dropna(inplace=True)

In [60]:
df.isna().sum()

Employee ID             0
Date of Joining         0
Gender                  0
Company Type            0
WFH Setup Available     0
Designation             0
Resource Allocation     0
Mental Fatigue Score    0
Burn Rate               0
dtype: int64

In [62]:
df.duplicated().sum()

np.int64(0)

In [63]:
x=df.drop(columns=['Employee ID', 'Burn Rate', 'Date of Joining'])
y=df[['Burn Rate']]

In [64]:
x.head()

,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
0,Female,Service,No,2.0,3.0,3.8
1,Male,Service,Yes,1.0,2.0,5.0
3,Male,Service,Yes,1.0,1.0,2.6
4,Female,Service,No,3.0,7.0,6.9
5,Male,Product,Yes,2.0,4.0,3.6


In [65]:
y.head()

,Burn Rate
0,0.16
1,0.36
3,0.20
4,0.52
5,0.29


In [66]:
cat=list(x.columns[x.dtypes=='object'])
con=list(x.columns[x.dtypes!='object'])

In [67]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [68]:
con_pipe=Pipeline(steps=[('impute', SimpleImputer(strategy='mean')),
                         ('scaler', StandardScaler())])

In [69]:
cat_pipe=Pipeline(steps=[('impute', SimpleImputer(strategy='constant')),
                         ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [70]:
pre = ColumnTransformer([('con', con_pipe, con),
                           ('cat', cat_pipe, cat)]).set_output(transform='pandas')

In [71]:
x_pre=pre.fit_transform(x)
x_pre.head()

,con__Designation,con__Resource Allocation,con__Mental Fatigue Score,cat__Gender_Female,cat__Gender_Male,cat__Company Type_Product,cat__Company Type_Service,cat__WFH Setup Available_No,cat__WFH Setup Available_Yes
0,-0.158322,-0.726994,-1.006081,1.0,0.0,0.0,1.0,1.0,0.0
1,-1.040844,-1.216041,-0.381242,0.0,1.0,0.0,1.0,0.0,1.0
3,-1.040844,-1.705088,-1.630920,0.0,1.0,0.0,1.0,0.0,1.0
4,0.724199,1.229194,0.608086,1.0,0.0,0.0,1.0,1.0,0.0
5,-0.158322,-0.237947,-1.110221,0.0,1.0,1.0,0.0,0.0,1.0


In [72]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_pre, y, test_size=0.2, random_state=21)

In [73]:
xtrain.shape

(14872, 9)

In [74]:
xtest.shape

(3718, 9)

In [75]:
xtrain.head()

,con__Designation,con__Resource Allocation,con__Mental Fatigue Score,cat__Gender_Female,cat__Gender_Male,cat__Company Type_Product,cat__Company Type_Service,cat__WFH Setup Available_No,cat__WFH Setup Available_Yes
12548,-1.040844,-1.705088,-0.901941,0.0,1.0,0.0,1.0,0.0,1.0
15230,-0.158322,0.740147,0.764296,1.0,0.0,0.0,1.0,0.0,1.0
10629,-1.040844,-0.726994,-0.537452,1.0,0.0,0.0,1.0,1.0,0.0
1189,-0.158322,-0.237947,0.399807,1.0,0.0,1.0,0.0,0.0,1.0
5144,-0.158322,-0.237947,-1.162290,1.0,0.0,0.0,1.0,1.0,0.0


In [76]:
ytrain.head()

,Burn Rate
12548,0.27
15230,0.51
10629,0.40
1189,0.48
5144,0.22


In [77]:
xtest.head()

,con__Designation,con__Resource Allocation,con__Mental Fatigue Score,cat__Gender_Female,cat__Gender_Male,cat__Company Type_Product,cat__Company Type_Service,cat__WFH Setup Available_No,cat__WFH Setup Available_Yes
5499,-0.158322,-0.237947,-0.641591,0.0,1.0,0.0,1.0,0.0,1.0
17718,-0.158322,-0.237947,0.243597,0.0,1.0,1.0,0.0,0.0,1.0
9042,-1.040844,-1.216041,0.139457,0.0,1.0,0.0,1.0,1.0,0.0
9435,0.724199,0.251100,0.191527,1.0,0.0,0.0,1.0,1.0,0.0
12823,0.724199,0.251100,0.347737,0.0,1.0,0.0,1.0,0.0,1.0


In [78]:
ytest.head()

,Burn Rate
5499,0.32
17718,0.39
9042,0.36
9435,0.47
12823,0.47


In [79]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(xtrain, ytrain)

LinearRegression()

In [80]:
lr.score(xtrain, ytrain)

0.9205332266666463

In [81]:
lr.score(xtest, ytest)

0.9210668311002596

In [82]:
ypred_train = lr.predict(xtrain)
ypred_test = lr.predict(xtest)

In [83]:
ypred_train[0:5]

array([[0.22729492],
       [0.59765625],
       [0.34362793],
       [0.48986816],
       [0.27856445]])

In [84]:
ytrain.head()

,Burn Rate
12548,0.27
15230,0.51
10629,0.40
1189,0.48
5144,0.22


In [85]:
ypred_test[0:5]

array([[0.34509277],
       [0.47192383],
       [0.4152832 ],
       [0.4942627 ],
       [0.50915527]])

In [86]:
ytest.head()

,Burn Rate
5499,0.32
17718,0.39
9042,0.36
9435,0.47
12823,0.47


In [87]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def evaluate_model(model, x, y):
    ypred = model.predict(x)
    mse = mean_squared_error(y, ypred)
    rmse = mse**(1/2)
    mae = mean_absolute_error(y, ypred)
    r2 = r2_score(y, ypred)
    print(f'Mean Squared Error : {mse:.2f}')
    print(f'Root Mean Squared Error : {rmse:.2f}')
    print(f'Mean Absolute Error : {mae:.2f}')
    print(f'R2 Score : {r2:.4f}')

In [88]:
evaluate_model(lr, xtrain, ytrain)

Mean Squared Error : 0.00
Root Mean Squared Error : 0.06
Mean Absolute Error : 0.05
R2 Score : 0.9205


In [89]:
evaluate_model(lr, xtest, ytest)

Mean Squared Error : 0.00
Root Mean Squared Error : 0.06
Mean Absolute Error : 0.05
R2 Score : 0.9211


In [90]:
xnew = pd.read_csv('test.csv')
xnew.head()

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
0,fffe31003300390039003000,2008-12-10,Female,Service,No,2.0,5.0,7.7
1,fffe31003300310037003800,2008-08-14,Female,Product,Yes,1.0,2.0,5.2
2,fffe33003400380035003900,2008-11-13,Male,Product,Yes,1.0,3.0,5.9
3,fffe3100370039003200,2008-02-07,Female,Service,No,3.0,6.0,4.6
4,fffe32003600390036003700,2008-07-17,Female,Product,No,2.0,5.0,6.4


In [91]:
pre

ColumnTransformer(transformers=[('con',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Designation', 'Resource Allocation',
                                  'Mental Fatigue Score']),
                                ('cat',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Gender', 'Company Type',
                                  'WFH Setup Available'])])

In [92]:
xnew_pre = pre.transform(xnew)
xnew_pre.head()

,con__Designation,con__Resource Allocation,con__Mental Fatigue Score,cat__Gender_Female,cat__Gender_Male,cat__Company Type_Product,cat__Company Type_Service,cat__WFH Setup Available_No,cat__WFH Setup Available_Yes
0,-0.158322,0.251100,1.024646,1.0,0.0,0.0,1.0,1.0,0.0
1,-1.040844,-1.216041,-0.277102,1.0,0.0,1.0,0.0,0.0,1.0
2,-1.040844,-0.726994,0.087387,0.0,1.0,1.0,0.0,0.0,1.0
3,0.724199,0.740147,-0.589522,1.0,0.0,0.0,1.0,1.0,0.0
4,-0.158322,0.251100,0.347737,1.0,0.0,1.0,0.0,1.0,0.0


In [93]:

preds = lr.predict(xnew_pre)
preds

array([[0.61755371],
       [0.34155273],
       [0.42614746],
       ...,
       [0.80944824],
       [0.5715332 ],
       [0.11157227]])

In [94]:
import numpy as np
pred1=np.round(preds, 2)
pred1


array([[0.62],
       [0.34],
       [0.43],
       ...,
       [0.81],
       [0.57],
       [0.11]])

In [95]:
df_final = xnew[['Employee ID']]
df_final

,Employee ID
0,fffe31003300390039003000
1,fffe31003300310037003800
2,fffe33003400380035003900
3,fffe3100370039003200
4,fffe32003600390036003700
...,...
12245,fffe3900310034003700
12246,fffe32003600330034003000
12247,fffe31003800340039003000
12248,fffe32003600380031003800


In [96]:
df_final['Burn Rate'] = pred1 

In [97]:
df_final

,Employee ID,Burn Rate
0,fffe31003300390039003000,0.62
1,fffe31003300310037003800,0.34
2,fffe33003400380035003900,0.43
3,fffe3100370039003200,0.41
4,fffe32003600390036003700,0.52
...,...,...
12245,fffe3900310034003700,0.41
12246,fffe32003600330034003000,0.45
12247,fffe31003800340039003000,0.81
12248,fffe32003600380031003800,0.57


In [98]:
df_final.to_csv('Predicted.csv', index=False)